### Coursera Capstone Project 

#### Importing all the required packages

In [1]:
import geocoder

In [2]:
from pandas.io.json import json_normalize 
from geopy.geocoders import Nominatim 
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import matplotlib.cm as cm
import pandas as pd 
import numpy as np
import geocoder
import requests 
import folium 
import json 
print("All libraries imported successfully")

All libraries imported successfully


### Scraping data from Wikipedia

In [3]:
data = requests.get("https://commons.wikimedia.org/wiki/Category:Suburbs_of_Bangalore").text
soup = BeautifulSoup(data, 'html.parser')
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
banglore_df = pd.DataFrame({"Neighborhood": neighborhoodList})
banglore_df.head()

,Neighborhood
0,"► Agara, Bangalore‎ (2 C, 6 F)"
1,► Arekere‎ (5 F)
2,"► Banashankari‎ (1 C, 4 F)"
3,► Banaswadi‎ (2 F)
4,"► Basavanagudi‎ (5 C, 11 F)"


In [4]:
new = banglore_df['Neighborhood'].str.split(' ', expand = True)

In [5]:
banglore_df['Neighborhood'] = new[2]

In [6]:
new = banglore_df['Neighborhood'].str.split(',',expand = True)

In [7]:
banglore_df['Neighborhood'] = new[0]

In [8]:
banglore_df.head()

,Neighborhood
0,Agara
1,Arekere‎
2,Banashankari‎
3,Banaswadi‎
4,Basavanagudi‎


In [9]:
banglore_df.shape

(59, 1)

### Getting coordinates for the cities

In [10]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Banglore, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
coordinates = [ get_latlng(neighborhood) for neighborhood in banglore_df["Neighborhood"].tolist() ]

In [12]:
df_coords = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])

In [13]:
banglore_df['Latitude'] = df_coords['Latitude']
banglore_df['Longitude'] = df_coords['Longitude']

In [14]:
# save the DataFrame as CSV file
banglore_df.to_csv("banglore_df.csv", index=False)

In [15]:
address = 'Banglore'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Banglore {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Banglore 12.9335752, 77.6885447.


In [16]:
map_banglore = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, neighborhood in zip(banglore_df['Latitude'], banglore_df['Longitude'], banglore_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_banglore)  
    
map_banglore

In [17]:
map_banglore.save('map_bangalore.html')

In [20]:
CLIENT_ID = 'Your Client ID' # your Foursquare ID
CLIENT_SECRET = 'Your Secret Key' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RE3GEXCG5HCQKT51QWR1VOXQQUB0HS5LTFNI1A4QJQHL5VOW
CLIENT_SECRET:WKFKMAVRI4NPGINIHDITKBXCKDP3OBVOTXOEVXDECNA50PRM


In [21]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(banglore_df['Latitude'], banglore_df['Longitude'], banglore_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [22]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3073, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Arekere‎,12.88568,77.59668,Decathlon Sports India Pvt Ltd,12.887513,77.597712,Sporting Goods Shop
1,Arekere‎,12.88568,77.59668,Natural Ice Cream,12.892188,77.598222,Ice Cream Shop
2,Arekere‎,12.88568,77.59668,Cinepolis,12.876119,77.595455,Multiplex
3,Arekere‎,12.88568,77.59668,Swensens,12.876071,77.595542,Ice Cream Shop
4,Arekere‎,12.88568,77.59668,Chavadi,12.892199,77.602538,BBQ Joint


In [23]:
banglore_df.head(10)

,Neighborhood,Latitude,Longitude
0,Agara,12.842830,77.487590
1,Arekere‎,12.885680,77.596680
2,Banashankari‎,12.873412,77.538013
3,Banaswadi‎,13.028477,77.631890
4,Basavanagudi‎,12.938980,77.571370
5,Begur,12.882450,77.624750
6,Bellandur‎,12.927350,77.671850
7,BEML‎,12.964900,77.592400
8,Bengaluru,12.971940,77.593690
9,Bidadi‎,12.830190,77.866230


In [24]:
venues_df.head(20)

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Arekere‎,12.88568,77.59668,Decathlon Sports India Pvt Ltd,12.887513,77.597712,Sporting Goods Shop
1,Arekere‎,12.88568,77.59668,Natural Ice Cream,12.892188,77.598222,Ice Cream Shop
2,Arekere‎,12.88568,77.59668,Cinepolis,12.876119,77.595455,Multiplex
3,Arekere‎,12.88568,77.59668,Swensens,12.876071,77.595542,Ice Cream Shop
4,Arekere‎,12.88568,77.59668,Chavadi,12.892199,77.602538,BBQ Joint
5,Arekere‎,12.88568,77.59668,Hakuna Matata,12.894028,77.586509,Lounge
6,Arekere‎,12.88568,77.59668,The Yellow Submarine,12.897626,77.599696,Indian Restaurant
7,Arekere‎,12.88568,77.59668,Ingu Tengu,12.883268,77.607514,South Indian Restaurant
8,Arekere‎,12.88568,77.59668,SLV Refreshments,12.890905,77.581927,Indian Restaurant
9,Arekere‎,12.88568,77.59668,The Pint Room,12.875989,77.595602,Beer Garden


#### Let's check how many venues were returned for each neighorhood

In [25]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Arekere‎,79,79,79,79,79,79
BEML‎,100,100,100,100,100,100
Banashankari‎,7,7,7,7,7,7
Banaswadi‎,54,54,54,54,54,54
Basavanagudi‎,100,100,100,100,100,100
Begur,12,12,12,12,12,12
Bellandur‎,84,84,84,84,84,84
Bengaluru,100,100,100,100,100,100
Bidadi‎,1,1,1,1,1,1


#### Finding unique categories in the venues_df

In [26]:
print("There are {} unique categories from venues".format(len(venues_df['VenueCategory'].unique())))

There are 206 unique categories from venues


#### Exploring the unique categories from venues_df

In [27]:
print(venues_df['VenueCategory'].unique())

['Sporting Goods Shop' 'Ice Cream Shop' 'Multiplex' 'BBQ Joint' 'Lounge'
 'Indian Restaurant' 'South Indian Restaurant' 'Beer Garden'
 'Bowling Alley' 'Shopping Mall' 'Office' 'Café' 'Pizza Place'
 'General Entertainment' 'Restaurant' 'Middle Eastern Restaurant'
 'Chinese Restaurant' 'Sandwich Place' 'Liquor Store' 'Department Store'
 'American Restaurant' 'Fast Food Restaurant'
 'Eastern European Restaurant' 'Coffee Shop' 'Burger Joint'
 'Rajasthani Restaurant' 'Supermarket' 'Italian Restaurant' 'Dive Bar'
 'Dumpling Restaurant' 'Dessert Shop' 'Electronics Store'
 'Badminton Court' 'Clothing Store' 'Gym' 'Breakfast Spot' 'Movie Theater'
 'Cosmetics Shop' 'Food Court' 'Vegetarian / Vegan Restaurant' 'Park'
 'Kebab Restaurant' 'Bakery' 'Convenience Store' 'Pharmacy' 'Donut Shop'
 'Trail' 'Korean Restaurant' 'Pub' 'Falafel Restaurant' 'Steakhouse'
 'Bistro' 'Diner' 'Snack Place' 'Indian Chinese Restaurant'
 'Kerala Restaurant' 'Brewery' 'Tea Room' 'Paper / Office Supplies Store'
 'Szechu

In [28]:
venues_df['VenueCategory'].value_counts()[:50]

Indian Restaurant                433
Café                             188
Ice Cream Shop                   141
Fast Food Restaurant             122
Pizza Place                      101
Hotel                             89
Coffee Shop                       89
Chinese Restaurant                82
Department Store                  67
Lounge                            61
Italian Restaurant                59
Restaurant                        55
Bakery                            54
Vegetarian / Vegan Restaurant     52
Breakfast Spot                    51
Pub                               49
Asian Restaurant                  46
Brewery                           40
Clothing Store                    38
Sandwich Place                    36
Burger Joint                      35
BBQ Joint                         33
Shopping Mall                     33
Snack Place                       31
Park                              31
Bar                               31
Donut Shop                        30
J

### Finding Best place to open a pub in Banglore

### Analyzing the neighborhoods information

#### One hot encoding for the neighboring 

In [29]:
# one hot encoding
banglore_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
banglore_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [banglore_onehot.columns[-1]] + list(banglore_onehot.columns[:-1])
banglore_onehot = banglore_onehot[fixed_columns]

print(banglore_onehot.shape)
banglore_onehot.head()

(3073, 207)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,...,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Arekere‎,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Arekere‎,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Arekere‎,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Arekere‎,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Arekere‎,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Grouping categories based on frequency

In [30]:
banglore_grouped = banglore_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(banglore_grouped.shape)
banglore_grouped.head(10)

(57, 207)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,...,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Arekere‎,0.0,0.0,0.000000,0.0,0.012658,0.00,0.00,0.00,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.025316,0.0,0.00,0.00,0.0
1,BEML‎,0.0,0.0,0.000000,0.0,0.010000,0.00,0.01,0.00,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.030000,0.0,0.01,0.00,0.0
2,Banashankari‎,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.00,0.0,...,0.142857,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.00,0.0
3,Banaswadi‎,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.00,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.018519,0.0,0.00,0.00,0.0
4,Basavanagudi‎,0.0,0.0,0.000000,0.0,0.000000,0.01,0.00,0.01,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.010000,0.0,0.00,0.01,0.0
5,Begur,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.00,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.00,0.0
6,Bellandur‎,0.0,0.0,0.011905,0.0,0.011905,0.00,0.00,0.00,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.00,0.0
7,Bengaluru,0.0,0.0,0.010000,0.0,0.010000,0.01,0.01,0.00,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.030000,0.0,0.01,0.00,0.0
8,Bidadi‎,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.00,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.00,0.0
9,Bommasandra‎,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.00,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.00,0.0


#### Checking number of places having pubs as VenueCategory

In [31]:
len(banglore_grouped[banglore_grouped["Pub"] > 0])

18

### Creating dataframe with pub data only

In [32]:
banglore_pub = banglore_grouped[["Neighborhoods","Pub"]]

In [33]:
banglore_pub.head()

,Neighborhoods,Pub
0,Arekere‎,0.000000
1,BEML‎,0.030000
2,Banashankari‎,0.000000
3,Banaswadi‎,0.018519
4,Basavanagudi‎,0.000000


### Creating clusters for the neighborhoods using Kmeans

In [34]:
# set number of clusters
kclusters = 5
banglore_clustering = banglore_pub.drop(["Neighborhoods"], 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(banglore_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 0, 2, 0, 0, 2, 1, 0, 0], dtype=int32)

In [35]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
banglore_merged = banglore_pub.copy()

# add clustering labels
banglore_merged["Cluster Labels"] = kmeans.labels_

In [36]:
banglore_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
banglore_merged.head()

,Neighborhood,Pub,Cluster Labels
0,Arekere‎,0.000000,0
1,BEML‎,0.030000,3
2,Banashankari‎,0.000000,0
3,Banaswadi‎,0.018519,2
4,Basavanagudi‎,0.000000,0


### Joinging dataframes to get the latitude and longitude for corresponding neighborhood

In [37]:
banglore_merged = banglore_merged.join(banglore_df.set_index("Neighborhood"), on="Neighborhood")

print(banglore_merged.shape)
banglore_merged.head(20) 

(57, 5)


,Neighborhood,Pub,Cluster Labels,Latitude,Longitude
0,Arekere‎,0.000000,0,12.885680,77.596680
1,BEML‎,0.030000,3,12.964900,77.592400
2,Banashankari‎,0.000000,0,12.873412,77.538013
3,Banaswadi‎,0.018519,2,13.028477,77.631890
4,Basavanagudi‎,0.000000,0,12.938980,77.571370
5,Begur,0.000000,0,12.882450,77.624750
6,Bellandur‎,0.011905,2,12.927350,77.671850
7,Bengaluru,0.040000,1,12.971940,77.593690
8,Bidadi‎,0.000000,0,12.830190,77.866230
9,Bommasandra‎,0.000000,0,12.817530,77.678790


### Sorting dataframe based on cluster labels

In [38]:
print(banglore_merged.shape)
banglore_merged.sort_values(["Cluster Labels"], inplace=True)
banglore_merged

(57, 5)


,Neighborhood,Pub,Cluster Labels,Latitude,Longitude
0,Arekere‎,0.000000,0,12.885680,77.596680
22,Ittamadu‎,0.000000,0,12.923440,77.542840
23,J.,0.000000,0,12.830190,77.866230
24,Jakkur‎,0.000000,0,13.075640,77.603940
25,Jayanagar,0.000000,0,12.928720,77.582810
26,Jeevanbheemanagar‎,0.000000,0,12.830190,77.866230
27,Kettohalli‎,0.000000,0,12.830190,77.866230
55,Yeshwantpur‎,0.000000,0,13.039120,77.577970
29,Konanakunte‎,0.000000,0,12.882330,77.569260
44,Ramamurthy,0.000000,0,12.951054,77.581482


### Mapping Clusters

In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(banglore_merged['Latitude'], banglore_merged['Longitude'], banglore_merged['Neighborhood'], banglore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [40]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### Examine Clusters

#### Cluster 1

In [42]:
banglore_merged.loc[banglore_merged['Cluster Labels'] == 0]

,Neighborhood,Pub,Cluster Labels,Latitude,Longitude
0,Arekere‎,0.0,0,12.885680,77.596680
22,Ittamadu‎,0.0,0,12.923440,77.542840
23,J.,0.0,0,12.830190,77.866230
24,Jakkur‎,0.0,0,13.075640,77.603940
25,Jayanagar,0.0,0,12.928720,77.582810
26,Jeevanbheemanagar‎,0.0,0,12.830190,77.866230
27,Kettohalli‎,0.0,0,12.830190,77.866230
55,Yeshwantpur‎,0.0,0,13.039120,77.577970
29,Konanakunte‎,0.0,0,12.882330,77.569260
44,Ramamurthy,0.0,0,12.951054,77.581482


In [48]:
len(banglore_merged.loc[banglore_merged['Cluster Labels'] == 0])

39

#### Cluster 2

In [43]:
banglore_merged.loc[banglore_merged['Cluster Labels'] == 1]

,Neighborhood,Pub,Cluster Labels,Latitude,Longitude
50,UB,0.04,1,12.973820,77.603100
47,Shivajinagar‎,0.04,1,12.987200,77.604010
28,Kodihalli,0.04,1,12.966200,77.649820
21,Indiranagar‎,0.04,1,12.978220,77.633970
10,Brigade,0.04,1,12.966236,77.606791
7,Bengaluru,0.04,1,12.971940,77.593690


#### Cluster 3

In [44]:
banglore_merged.loc[banglore_merged['Cluster Labels'] == 2]

,Neighborhood,Pub,Cluster Labels,Latitude,Longitude
42,Rajajinagar‎,0.013158,2,13.011920,77.547170
37,Malleswaram‎,0.010000,2,13.006322,77.568416
30,Koramangala‎,0.010000,2,12.920040,77.625460
45,Sahakara,0.017241,2,12.923863,77.548150
18,HSR,0.013699,2,12.919430,77.649130
6,Bellandur‎,0.011905,2,12.927350,77.671850
3,Banaswadi‎,0.018519,2,13.028477,77.631890
56,Yeswanthpur‎,0.010204,2,13.018644,77.557617


#### Cluster 4

In [45]:
banglore_merged.loc[banglore_merged['Cluster Labels'] == 3]

,Neighborhood,Pub,Cluster Labels,Latitude,Longitude
1,BEML‎,0.03,3,12.9649,77.5924


#### Cluster 5

In [46]:
banglore_merged.loc[banglore_merged['Cluster Labels'] == 4]

,Neighborhood,Pub,Cluster Labels,Latitude,Longitude
49,Tippasandra‎,0.050000,4,12.97171,77.64780
51,Ulsoor‎,0.050000,4,12.98908,77.62795
40,Murugeshpalya‎,0.047619,4,12.95565,77.65335


#### Observations

Out of all the cities in Banglore, we have made 5 clusters using KMeans Clustering Algorithm. As we can see that first cluster has 39 datapoints, Cluster 2 has 6 datapoints, Cluster 3 has 8 datapoints, Cluster 4 has only one datapoint(i.e. BEML), whereas Cluster 5 has 3 datapoints. Clearly from the data, we can say that Cluster 4 is good for opening a Pub.